In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0119 22:32:03.404707  6012 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0119 22:32:03.409180  6012 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='c_rnn'
start_checkpoint=None
logging_interval=10
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='10000,10000,10000' #Declare  the training steps for which the learning rates will be used
batch_size=256
model_size_info=[48, 10, 4, 2, 2, 2, 60, 84]

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 22:33:40.506734  6012 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 22:33:40.554772  6012 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [9]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [8]:
# def get_test_data(args):
#     '''
#     Input: (sess,offset)
#     '''
#     sess,i=args
#     test_data, test_ground_truth = audio_processor.get_data(
#         batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
#     return test_data,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      data_input,
      model_settings,
      model_architecture,
      model_size_info=model_size_info,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [11]:
tf.app.run(main=main)

W0119 22:35:00.436802  6012 deprecation_wrapper.py:119] From ..\libs\models.py:676: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0119 22:35:00.506020  6012 deprecation.py:506] From ..\libs\models.py:686: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0119 22:35:00.551414  6012 deprecation.py:323] From ..\libs\models.py:710: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W0119 22:35:00.555913  6012 deprecation.py:323] From ..\libs\models.py:721: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
I

I0119 22:37:45.161154  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-30"
I0119 22:37:49.844935  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-31"
I0119 22:37:54.757374  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-32"
I0119 22:38:00.972667  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-33"
I0119 22:38:07.304586  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-34"
I0119 22:38:12.344317  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-35"
I0119 22:38:18.226715  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-36"
I0119 22:38:22.811984  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-37"
I0119 22:38:27.921539  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-38"
I0119 22:38:32.907968  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-39"
I0119 22:38:39.021674  6012 trainer.py:126] Step #40: rate 0.000500, accuracy 4.

I0119 22:44:23.779629  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-110"
I0119 22:44:28.457030  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-111"
I0119 22:44:32.992314  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-112"
I0119 22:44:37.564036  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-113"
I0119 22:44:42.986406  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-114"
I0119 22:44:48.374340  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-115"
I0119 22:44:53.529022  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-116"
I0119 22:44:56.790698  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-117"
I0119 22:44:59.881897  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-118"
I0119 22:45:04.083647  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-119"
I0119 22:45:08.908997  6012 trainer.py:126] Step #120: rate 0.000500, 

I0119 22:51:01.012243  6012 trainer.py:126] Step #190: rate 0.000500, accuracy 16.8%, cross entropy 2.736160
I0119 22:51:01.014657  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-190"
I0119 22:51:05.904808  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-191"
I0119 22:51:09.784671  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-192"
I0119 22:51:13.899919  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-193"
I0119 22:51:18.471529  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-194"
I0119 22:51:23.229620  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-195"
I0119 22:51:27.532378  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-196"
I0119 22:51:32.156556  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-197"
I0119 22:51:37.084762  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-198"
I0119 22:51:41.905025  6012 trainer.py:172] Saving to 

I0119 22:56:05.835966  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-269"
I0119 22:56:07.928262  6012 trainer.py:126] Step #270: rate 0.000500, accuracy 19.5%, cross entropy 2.525553
I0119 22:56:07.931258  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-270"
I0119 22:56:09.588891  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-271"
I0119 22:56:11.268898  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-272"
I0119 22:56:12.957851  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-273"
I0119 22:56:14.559805  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-274"
I0119 22:56:16.120890  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-275"
I0119 22:56:17.821061  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-276"
I0119 22:56:19.290906  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-277"
I0119 22:56:20.711003  6012 trainer.py:172] Saving to 

I0119 22:58:46.915013  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-348"
I0119 22:58:49.054489  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-349"
I0119 22:58:51.214945  6012 trainer.py:126] Step #350: rate 0.000500, accuracy 27.7%, cross entropy 2.335005
I0119 22:58:51.215942  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-350"
I0119 22:58:53.287816  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-351"
I0119 22:58:55.447067  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-352"
I0119 22:58:57.581935  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-353"
I0119 22:58:59.675806  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-354"
I0119 22:59:01.977072  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-355"
I0119 22:59:04.121368  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-356"
I0119 22:59:06.301650  6012 trainer.py:172] Saving to 

I0119 23:02:02.421061  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-427"
I0119 23:02:04.923951  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-428"
I0119 23:02:07.260337  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-429"
I0119 23:02:09.481889  6012 trainer.py:126] Step #430: rate 0.000500, accuracy 31.2%, cross entropy 2.115963
I0119 23:02:09.482887  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-430"
I0119 23:02:11.771056  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-431"
I0119 23:02:13.985322  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-432"
I0119 23:02:16.190936  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-433"
I0119 23:02:18.265359  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-434"
I0119 23:02:20.444350  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-435"
I0119 23:02:22.649499  6012 trainer.py:172] Saving to 

I0119 23:04:51.641826  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-506"
I0119 23:04:53.650051  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-507"
I0119 23:04:58.939565  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-508"
I0119 23:05:02.162012  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-509"
I0119 23:05:04.658967  6012 trainer.py:126] Step #510: rate 0.000500, accuracy 35.2%, cross entropy 1.931417
I0119 23:05:04.660949  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-510"
I0119 23:05:06.500073  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-511"
I0119 23:05:08.439118  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-512"
I0119 23:05:10.375015  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-513"
I0119 23:05:12.232542  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-514"
I0119 23:05:14.128453  6012 trainer.py:172] Saving to 

I0119 23:07:32.414404  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-585"
I0119 23:07:34.233016  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-586"
I0119 23:07:36.155863  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-587"
I0119 23:07:38.049628  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-588"
I0119 23:07:39.894791  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-589"
I0119 23:07:44.270646  6012 trainer.py:126] Step #590: rate 0.000500, accuracy 41.0%, cross entropy 1.844936
I0119 23:07:44.271644  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-590"
I0119 23:07:46.943041  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-591"
I0119 23:07:49.249999  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-592"
I0119 23:07:51.157676  6012 trainer.py:172] Saving to "..\..\logs&checkpoint\c_rnn\ckpt-593"
I0119 23:07:53.053134  6012 trainer.py:172] Saving to 

KeyboardInterrupt: 

In [ ]:
# save_checkpoint='..\\..\\logs&checkpoint\\c_rnn\\ckpt-42000'

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)